In [1]:
from __future__ import annotations

# Import the ChatGroq class from the langchain_groq package
from langchain_groq import ChatGroq
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage

from typing import TypedDict, List, Annotated, Literal
from pydantic import BaseModel, Field
from langgraph.graph import StateGraph, START, END
from langgraph.types import Send


In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
class Task(BaseModel):
    id: int
    title: str

    goal: str = Field(
        ...,
        description="One sentence describing what the reader should be able to do/understand after this section.",
    )
    bullets: List[str] = Field(
        ...,
        min_length=3,
        max_length=5,
        description="3-5 concrete, non-overlapping subpoints to cover in this section.",
    )
    target_words: int = Field(
        ...,
        description="Target word count for this section (120–450).",
    )
    section_type: Literal[
        "intro", "core", "examples", "checklist", "common_mistakes", "conclusion"
    ] = Field(
        ...,
        description="Use 'common_mistakes' exactly once in the plan.",
    )

In [4]:
class Plan(BaseModel):
    blog_title: str
    audience: str = Field(..., description="Who this blog is for.")
    tone: str = Field(..., description="Writing tone (e.g., practical, crisp).")
    tasks: List[Task]
    

In [5]:
import operator

class State(TypedDict):
    topic: str
    plan: Plan
    sections: Annotated[List[str], operator.add]  # reducer concatenates worker outputs
    final: str

In [6]:
llm = ChatGroq(model="llama-3.3-70b-versatile")


In [7]:
llm.invoke("Good Morning")

AIMessage(content='Good morning! How can I assist you today?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 37, 'total_tokens': 48, 'completion_time': 0.021660729, 'completion_tokens_details': None, 'prompt_time': 0.000945394, 'prompt_tokens_details': None, 'queue_time': 0.050576966, 'total_time': 0.022606123}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_c06d5113ec', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019c6516-dc72-73d0-97f7-c94d0a3d7a6d-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 37, 'output_tokens': 11, 'total_tokens': 48})

In [8]:
def orchestrator(state: State):
    plan = llm.with_structured_output(Plan).invoke(
        [
            SystemMessage(
                content=(
                    "You are an experienced technical writer who creates clear, friendly, and engaging blog plans. "
                    "Your audience includes recruiters, hiring managers, and professionals who prefer concise, readable content. "
                    "Avoid jargon-heavy language. Focus on clarity, real-world relevance, and logical flow."
                )
            ),
            HumanMessage(
                content=(
                    f"Create a blog outline with 6-7 well-structured sections on the topic below.\n\n"
                    f"Topic: {state['topic']}\n\n"
                    "Return a plan that matches this structure:\n"
                    "- blog_title: clear, engaging\n"
                    "- audience: who this blog is for\n"
                    "- tone: writing tone (e.g., practical, crisp)\n"
                    "- tasks: list of sections with fields:\n"
                    "  - id: integer\n"
                    "  - title: section title\n"
                    "  - goal: one-sentence outcome for the reader\n"
                    "  - bullets: 3-5 concrete, non-overlapping subpoints\n"
                    "  - target_words: 120-450\n"
                    "  - section_type: one of intro, core, examples, checklist, common_mistakes, conclusion\n"
                    "Use 'common_mistakes' exactly once in the tasks list.\n\n"
                    "Tone should be friendly, conversational, and practical. The outline should feel approachable, professional, and easy to skim."
                )
            ),
        ]
    )
    return {"plan": plan}

In [9]:
# Visualize(orchestrator)

In [10]:
plan = llm.with_structured_output(Plan).invoke(
    [
        SystemMessage(
            content=(
                "You are an experienced technical writer who creates clear, friendly, and engaging blog plans. "
                "Your audience includes recruiters, hiring managers, and professionals who prefer concise, readable content. "
                "Avoid jargon-heavy language. Focus on clarity, real-world relevance, and logical flow."
            )
        ),
        HumanMessage(
            content=(
                f"Create a blog outline with 6-7 well-structured sections on the topic below.\n\n"
                f"Topic: agentic AI \n\n"
                "Return a plan that matches this structure:\n"
                "- blog_title: clear, engaging\n"
                "- audience: who this blog is for\n"
                "- tone: writing tone (e.g., practical, crisp)\n"
                "- tasks: list of sections with fields:\n"
                "  - id: integer\n"
                "  - title: section title\n"
                "  - goal: one-sentence outcome for the reader\n"
                "  - bullets: 3-5 concrete, non-overlapping subpoints\n"
                "  - target_words: 120-450\n"
                "  - section_type: one of intro, core, examples, checklist, common_mistakes, conclusion\n"
                "Use 'common_mistakes' exactly once in the tasks list.\n\n"
                "Tone should be friendly, conversational, and practical. The outline should feel approachable, professional, and easy to skim."
            )
        ),
    ]
)



In [11]:
plan.blog_title

'Unlocking the Power of Agentic AI: A Guide to Smarter Decision-Making'

In [12]:
plan.tasks

[Task(id=1, title='Introduction to Agentic AI', goal='The reader will understand the concept of agentic AI and its significance in modern technology', bullets=['Definition of agentic AI', 'Brief history and evolution of agentic AI', 'Importance of agentic AI in various industries'], target_words=200, section_type='intro'),
 Task(id=2, title='Core Principles of Agentic AI', goal='The reader will be able to explain the core principles of agentic AI and how they enable decision-making', bullets=['Autonomy and self-directed learning', 'Goal-oriented behavior and decision-making', 'Adaptability and resilience in complex environments'], target_words=300, section_type='core'),
 Task(id=3, title='Real-World Applications of Agentic AI', goal='The reader will be able to identify potential applications of agentic AI in their industry or field', bullets=['Healthcare and medical diagnosis', 'Financial analysis and portfolio management', 'Cybersecurity and threat detection'], target_words=250, secti

In [13]:
def task_distributor(state: State):
    """
    Dynamically creates one worker per task.
    The number of workers equals the number of sections in the plan.
    """

    tasks = state["plan"].tasks

    # Create one Send event per task
    sends = [
        Send(
            "section_writer",  # name of the worker node
            {
                "topic": state["topic"],
                "task": task,  # pass individual task
                "plan": state["plan"],
            },
        )
        for task in tasks
    ]

    return sends

In [14]:
def worker(payload: dict):
    task = payload["task"]
    plan = payload["plan"]
    topic = payload["topic"]

    bullets_text = "\n".join([f"- {item}" for item in task.bullets])

    # write the section
    section_md = llm.invoke(
        [
            SystemMessage(
                content=(
                    "Write one clean markdown section for a professional blog post."
                )
            ),
            HumanMessage(
                content=(
                    f"Write a blog section on the topic below in markdown only.\n\n"
                    f"Blog: {plan.blog_title}\n\n"
                    f"Audience: {plan.audience}\n"
                    f"Tone: {plan.tone}\n\n"
                    f"Topic: {topic}\n\n"
                    f"Section: {task.title}\n"
                    f"Section type: {task.section_type}\n"
                    f"Goal: {task.goal}\n"
                    f"Target words: {task.target_words}\n\n"
                    "Must cover:\n"
                    f"{bullets_text}\n\n"
                    "Write in a clear, friendly, and practical voice. Avoid fluff and keep the section self-contained."
                )
            ),
        ]
    ).content.strip()

    return {"sections": [section_md]}



In [15]:
from pathlib import Path


def finalizer(state: State):
    title = state["plan"].blog_title
    sections = state["sections"]
    final_md = f"# {title}\n\n" + "\n\n".join(sections)
    
    # save file
    file_name = title.lower().replace(" ", "_").replace(":", "").replace("'", "") + ".md"
    Path(file_name).write_text(final_md, encoding="utf-8")
    
    return {"final": final_md}


In [16]:
graph = StateGraph(State)


graph.add_node("orchestrator", orchestrator)
graph.add_node("section_writer", worker)
graph.add_node("finalizer", finalizer)

graph.add_edge(START, "orchestrator")
graph.add_conditional_edges(
    "orchestrator",
    task_distributor,
    ["section_writer"]
)

graph.add_edge("section_writer", "finalizer")
graph.add_edge("finalizer", END)

app = graph.compile()

In [19]:
blog = app.invoke({"topic": "langgraph and langchain for building agentic ai system","sections":[]})

In [18]:
blog

{'topic': 'langgraph',
 'plan': Plan(blog_title='Unlocking the Power of LangGraph: A Comprehensive Guide', audience='recruiters, hiring managers, and professionals', tone='friendly, conversational, and practical', tasks=[Task(id=1, title='Introduction to LangGraph', goal='The reader will understand the basic concept of LangGraph and its significance.', bullets=['Definition and explanation of LangGraph', 'Importance of LangGraph in natural language processing', 'Brief overview of LangGraph applications'], target_words=200, section_type='intro'), Task(id=2, title='Core Concepts of LangGraph', goal='The reader will be able to explain the core components and functionality of LangGraph.', bullets=['Graph structure and node relationships', 'Edge types and their roles in LangGraph', 'Node embeddings and their applications'], target_words=300, section_type='core'), Task(id=3, title='Real-World Applications of LangGraph', goal='The reader will understand the practical uses of LangGraph in vario